# Continuous Tomography

In [1]:
import tomobase
import numpy as np
import tomobase.phantoms
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
vol = tomobase.phantoms.nanocage()


2024-11-25 12:10:58,524 - DEBUG - This is a debug message


In [63]:

import tomobase.processes
import tomobase.processes.alignments
import tomobase.tiltschemes
import stackview


scheme = tomobase.tiltschemes.Incremental(-64, 64, 2)
angles = np.array([scheme.get_angle() for i in range(64)])
ts = tomobase.processes.project(vol, angles)
ts_noise = tomobase.processes.alignments.add_noise(ts, 0, 0, inplace=False)

ts.show(binning=1)
ts_noise.show(binning=1)
print(np.any(ts_noise.data < 0))

False


In [64]:
from skimage.filters import threshold_otsu
#rec1 = tomobase.processes.reconstruct(ts, method="sirt", iterations=100)
rec2 = tomobase.processes.reconstruct(ts_noise, method="sirt", iterations=100)
ssim = structural_similarity(vol.data, rec2.data, data_range=1.0)
psnr = peak_signal_noise_ratio(vol.data, rec2.data, data_range =1.0)
print(ssim, psnr)


rec2.show(binning=1)
vol.show(binning=1)

2024-11-25 10:55:31,947 - INFO - type of volume: <class 'tomobase.data.volume.Volume'>
INFO:tomobase_logger:type of volume: <class 'tomobase.data.volume.Volume'>


0.8503745976376718 22.19590534557192


In [ ]:
num = 65
ssim = [0]*num
psnr = [0]*num
for i in range(num):
    scheme = tomobase.tiltschemes.Incremental(-64, 64, 2)
    angles = np.array([scheme.get_angle() for j in range(i+1)])
    ts = tomobase.processes.project(vol, angles)
    ts_noise = tomobase.processes.alignments.add_noise(ts, 0, 20, inplace=False)
    
    rec = tomobase.processes.reconstruct(ts_noise, method="sirt", iterations=100)
    ssim[i] = structural_similarity(vol.data, rec.data, data_range=1.0)
    psnr[i] = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)
    print(ssim[i], psnr[i])

print(i,ssim, psnr)





In [ ]:
for i in range(65):
    print(psnr[i])

#import stackview

#stackview.curtain(vol.data, rec.data)

In [27]:


max_angle = 64
min_angle = -64
n_images=0
k= 2
step =  (max_angle - min_angle)/(k+0.5)


offset=0
offset_set = 2
angles= [None]*65
isforward = True
for i in range(65):
    if i == 0:
        angles[i] = min_angle
    else:
        if (angles[i-1] + step)>= max_angle:
            if (offset+0.5)>=1:
                if offset == (offset_set-1)/offset_set:
                    offset_set = offset_set*2
                    offset_run = 1/offset_set
                    offset=offset_run
                else:
                    offset_run += 2/offset_set
                    offset = offset_run  
            else:
                offset += 0.5
            angles[i] = min_angle+(offset*step)
        else:
            angles[i] = angles[i-1]+step
    #print( angles[i])

max_angle = 64
min_angle = -64
n_images=0
k= 4
step =  (max_angle - min_angle)/(k+0.5)


offset=0
offset_set = 2
angles= [None]*65
isforward = True
max_cutoff = (max_angle)-(step/2)
for i in range(65):
    if i == 0:
        angles[i] = min_angle
    else:
        if isforward:
            if (angles[i-1] + step)>= max_angle:
                if (offset+0.5)>=1:
                    if offset == (offset_set-1)/offset_set:
                        offset_set = offset_set*2
                        offset_run = 1/offset_set
                        offset=offset_run
                    else:
                        offset_run += 2/offset_set
                        offset = offset_run  
                else:
                    offset += 0.5
                isforward = False
                if (max_cutoff + (step*offset))>=max_angle:
                    angles[i] = max_cutoff + (step*offset) - step
                else:
                    angles[i] = max_cutoff + (step*offset)
                step *= -1
            else:
                angles[i] = angles[i-1]+step
        else:
            if (angles[i-1] + step)<= min_angle:
                if (offset+0.5)>=1:
                    if offset == (offset_set-1)/offset_set:
                        offset_set = offset_set*2
                        offset_run = 1/offset_set
                        offset=offset_run
                    else:
                        offset_run += 2/offset_set
                        offset = offset_run  
                else:
                    offset += 0.5
                isforward = True
                angles[i] = min_angle+(offset*np.abs(step))
                step *= -1
            else:
                angles[i] = angles[i-1]+step
    print( angles[i])


-64
-35.55555555555556
-7.111111111111114
21.33333333333333
49.77777777777777
35.55555555555556
7.111111111111114
-21.33333333333333
-49.77777777777777
-56.888888888888886
-28.444444444444443
0.0
28.444444444444443
56.888888888888886
42.66666666666667
14.222222222222229
-14.222222222222214
-42.66666666666666
-60.44444444444444
-32.0
-3.555555555555557
24.888888888888886
53.33333333333333
39.111111111111114
10.666666666666671
-17.77777777777777
-46.222222222222214
-53.333333333333336
-24.888888888888893
3.55555555555555
31.999999999999993
60.444444444444436
46.222222222222214
17.77777777777777
-10.666666666666671
-39.111111111111114
-62.22222222222222
-33.77777777777778
-5.333333333333336
23.111111111111107
51.55555555555555
37.33333333333333
8.888888888888886
-19.555555555555557
-48.0
-58.666666666666664
-30.22222222222222
-1.7777777777777786
26.666666666666664
55.11111111111111
40.888888888888886
12.444444444444443
-16.0
-44.44444444444444
-55.111111111111114
-26.66666666666667
1.7777